In [ ]:
import requests
import base64
import json
import pandas
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import Session
from datetime import datetime, timedelta
import pytz


In [ ]:
session = get_active_session()

In [ ]:
use schema python_workloads.genesys

In [ ]:
# Get Genesys ID and Secret

from snowflake.snowpark.context import get_active_session
# Snowflake session initialization
session = get_active_session()

client_key_query = "select python_workloads.genesys.get_client_key()"
client_key = session.sql(client_key_query).collect()[0][0]


client_secret_query = "select python_workloads.genesys.get_client_secret()"
client_secret = session.sql(client_secret_query).collect()[0][0]


In [ ]:
# https://developer.genesys.cloud/authorization/platform-auth/base-64-encoding

encodedData = base64.b64encode(bytes(f"{client_key}:{client_secret}", "ISO-8859-1")).decode("ascii")
authorization_header_string = f"Authorization: Basic {encodedData}"
print(authorization_header_string)
print('client_key : ', client_key)
print('client_secret : ', client_secret)

In [ ]:
# Get Bearer Token

# https://developer.genesys.cloud/authorization/platform-auth/use-client-credentials

token_endpoint = 'https://login.mypurecloud.com/oauth/token'

headers = {"Content-Type":"application/x-www-form-urlencoded",
           "Authorization": "Basic {0}".format(encodedData)
          
}


post_data = {
    "grant_type" : "client_credentials"
}

# print(headers)

response = requests.post(token_endpoint, data=post_data, headers=headers)

print(response.text)
print(response.content)
response_dic=response.json()
auth_token=response_dic['access_token']

In [ ]:
def fetch_data(url, token, query_payload=False):
    api_endpoint=url 
    headers = {"Content-Type":"application/json",
           "Authorization": "Bearer {0}".format(token)
          }
    if query_payload:
        response=requests.post(api_endpoint, headers=headers, data=json.dumps(query_payload))
    else:
        response=requests.get(api_endpoint, headers=headers)
    response_json=response.json()
    return response_json

In [ ]:
#users api (i guess this is agents?)

table_name='genesys.raw.agent_json'
#initial endpoint
page_size=100
page_number=1
base_url='https://api.mypurecloud.com'
endpoint='/api/v2/users?pageSize={0}&pageNumber={1}'.format(page_size, page_number)
api_endpoint = '{0}{1}'.format(base_url, endpoint)
#flag to stop pagination
keep_paginating=True 
#list to collect all data from each pagination api hit
all_data=[]
#pagination loop
while keep_paginating:
    users_data=fetch_data(api_endpoint, auth_token)
    all_data.extend(users_data['entities'])
    print(users_data)
    print('entity keys ',users_data['entities'][0].keys())
    if 'nextUri' in users_data.keys():
        api_endpoint = '{0}{1}'.format(base_url, users_data['nextUri'])
    else:
        keep_paginating=False  
print(all_data)
print(len(all_data))
df=pandas.DataFrame({'json_data':all_data})
snowflake_df=session.create_dataframe(df)
#truncate staging table and load data
session.sql(f"truncate table {table_name}").collect()
snowflake_df.write.mode("append").save_as_table(table_name)
    
    
    
    

In [ ]:
create or replace table genesys.transform.agents as
with address_data as (
select base.json_data:id::varchar(100) as userid
,addr.value:mediaType::varchar(100) as mediatype -- 'EMAIL', 'PHONE', 'SMS'
,addr.value:type::varchar(100) as type --EMAIL = 'WORK', PHONE = 'WORK', SMS = 'MOBILE' OR 'WORK'
,addr.value:address::varchar(100) as address
from genesys.raw.agent_json base,
lateral flatten (input=>base.json_data:addresses) addr
)
, primary_contact_data as (
select base.json_data:id::varchar(100) as userid
,pcd.value:mediaType::varchar(100) as mediatype -- 'EMAIL', 'PHONE', 'SMS' ALL TYPE = 'PRIMARY'
,pcd.value:type::varchar(100) as type
,pcd.value:address::varchar(100) as address
from genesys.raw.agent_json base,
lateral flatten (input=>base.json_data:primaryContactInfo) pcd
)

, base_attributes as (
select  
base.json_data
,base.json_data:id::varchar(100) as userid
,base.json_data:name::varchar(100) as name
,base.json_data:division.id::varchar(100) as division_id
,base.json_data:division.name::varchar(100) as division_name
,base.json_data:chat.jabberId::varchar(100) as chat_jabber_id--dont need 
,base.json_data:email::varchar(100) as email
-- ,base.json_data:primaryContactInfo 
-- ,base.json_data:addresses --"mediaType": "PHONE",
,base.json_data:state::varchar(100) as state
,base.json_data:username::varchar(100) as username
,base.json_data:version as version
,base.json_data:acdAutoAnswer as acdautoanswer
,base.json_data:selfUri::varchar(100) as selfuri
from genesys.raw.agent_json base
)
select b.userid
,b.name
,b.username
,b.selfuri
,b.state
,b.division_id
,b.division_name
,b.email
,a_work_email.address as work_email
,pcd_primary_email.address as primary_email
,a_work_phone.address as work_phone
,pcd_primary_phone.address as primary_phone
,a_work_sms.address as work_sms
,a_mobile_sms.address as mobile_sms
,pcd_primary_sms.address as primary_sms
,b.chat_jabber_id

,b.version
,b.acdautoanswer

from base_attributes b 
left join address_data a_work_email on b.userid=a_work_email.userid and a_work_email.mediatype='EMAIL' and a_work_email.type='WORK'
left join address_data a_work_phone on b.userid=a_work_phone.userid and a_work_phone.mediatype='PHONE' and a_work_phone.type='WORK'
left join address_data a_work_sms on b.userid=a_work_sms.userid and a_work_sms.mediatype='SMS' and a_work_sms.type='WORK'
left join address_data a_mobile_sms on b.userid=a_mobile_sms.userid and a_mobile_sms.mediatype='SMS' and a_mobile_sms.type='MOBILE'
left join primary_contact_data pcd_primary_email on b.userid=pcd_primary_email.userid and pcd_primary_email.mediatype='EMAIL' and pcd_primary_email.type='PRIMARY'
left join primary_contact_data pcd_primary_phone on b.userid=pcd_primary_phone.userid and pcd_primary_phone.mediatype='PHONE' and pcd_primary_phone.type='PRIMARY'
left join primary_contact_data pcd_primary_sms on b.userid=pcd_primary_sms.userid and pcd_primary_sms.mediatype='SMS' and pcd_primary_sms.type='PRIMARY'

In [ ]:
#skills api. There are only 70 records for this. it looks like a reference table
#not actively running because we needed queues not skills for inital reporting

#initial endpoint
page_size=100
page_number=1
base_url='https://api.mypurecloud.com'
endpoint='/api/v2/routing/skills?pageSize={0}&pageNumber={1}'.format(page_size, page_number)
api_endpoint = '{0}{1}'.format(base_url, endpoint)
#flag to stop pagination
keep_paginating=True 
#list to collect all data from each pagination api hit
all_skills_data=[]
#pagination loop
# while keep_paginating:
#     skills_data=fetch_data(api_endpoint, auth_token)
#     all_skills_data.extend(skills_data['entities'])
#     print(skills_data)
#     print(skills_data.keys())
#     print('entity keys ',skills_data['entities'][0].keys())
#     if 'nextUri' in skills_data.keys():
#         api_endpoint = '{0}{1}'.format(base_url, skills_data['nextUri'])
#     else:
#         keep_paginating=False  
# print(all_skills_data)
# print(len(all_skills_data))
# for i in all_skills_data: 
#     print(i)

In [ ]:
#skills api. There are only 70 records for this. it looks like a reference table

table_name='genesys.raw.queue_json'
#initial endpoint
page_size=100
page_number=1
base_url='https://api.mypurecloud.com'
endpoint='/api/v2/routing/queues?pageSize={0}&pageNumber={1}'.format(page_size, page_number)
api_endpoint = '{0}{1}'.format(base_url, endpoint)
#flag to stop pagination
keep_paginating=True 
#list to collect all data from each pagination api hit
all_queue_data=[]
#pagination loop
while keep_paginating:
    queue_data=fetch_data(api_endpoint, auth_token)
    all_queue_data.extend(queue_data['entities'])
    print(queue_data)
    print(queue_data.keys())
    print('entity keys ',queue_data['entities'][0].keys())
    if 'nextUri' in queue_data.keys():
        api_endpoint = '{0}{1}'.format(base_url, queue_data['nextUri'])
    else:
        keep_paginating=False  
print(all_queue_data)
print(len(all_queue_data))
for i in all_queue_data: 
    print(i)

df=pandas.DataFrame({'json_data':all_queue_data})
snowflake_df=session.create_dataframe(df)
#truncate staging table and load data
session.sql(f"truncate table {table_name}").collect()
snowflake_df.write.mode("append").save_as_table(table_name)

In [ ]:
create or replace table genesys.transform.queues as
select 
base.json_data:id::varchar(100) as queueid
,base.json_data:name::varchar(100) as name
,base.json_data:division.id::varchar(100) as division_id
,base.json_data:division.name::varchar(100) as division_name
,base.json_data:dateCreated::timestamp_ntz as date_created
,base.json_data:dateModified::timestamp_ntz as date_modified
,base.json_data:modifiedBy::varchar(100) as modifiedby
,base.json_data:createdBy::varchar(100) as createdby
,base.json_data:memberCount as member_count
,base.json_data:userMemberCount as user_member_count
,base.json_data:joinedMemberCount as joined_member_count

,base.json_data:mediaSettings.call.alertingTimeoutSeconds as mediasettings_call_alertingtimeoutseconds
,base.json_data:mediaSettings.call.serviceLevel.durationMs as mediasettings_call_servicelevel_durationMs
,base.json_data:mediaSettings.call.serviceLevel.percentage::float as mediasettings_call_servicelevel_percentage

,base.json_data:mediaSettings.callback.alertingTimeoutSeconds as mediasettings_callback_alertingtimeoutseconds
,base.json_data:mediaSettings.callback.autoEndDelaySeconds as mediasettings_callback_autoEndDelaySeconds
,base.json_data:mediaSettings.callback.enableAutoDialAndEnd as mediasettings_callback_enableAutoDialAndEnd
,base.json_data:mediaSettings.callback.serviceLevel.durationMs as mediasettings_callback_servicelevel_durationMs
,base.json_data:mediaSettings.callback.serviceLevel.percentage::float as mediasettings_callback_servicelevel_percentage

,base.json_data:mediaSettings.chat.alertingTimeoutSeconds as mediasettings_chat_alertingtimeoutseconds
,base.json_data:mediaSettings.chat.serviceLevel.durationMs as mediasettings_chat_servicelevel_durationMs
,base.json_data:mediaSettings.chat.serviceLevel.percentage::float as mediasettings_chat_servicelevel_percentage

,base.json_data:mediaSettings.email.alertingTimeoutSeconds as mediasettings_email_alertingtimeoutseconds
,base.json_data:mediaSettings.email.serviceLevel.durationMs as mediasettings_email_servicelevel_durationMs
,base.json_data:mediaSettings.email.serviceLevel.percentage::float as mediasettings_email_servicelevel_percentage

,base.json_data:mediaSettings.message.alertingTimeoutSeconds as mediasettings_message_alertingtimeoutseconds
,base.json_data:mediaSettings.message.serviceLevel.durationMs as mediasettings_message_servicelevel_durationMs
,base.json_data:mediaSettings.message.serviceLevel.percentage::float as mediasettings_message_servicelevel_percentage

,base.json_data:scoringMethod::varchar(100) as scoringMethod
,base.json_data:acwSettings.timeoutMs as acwsettings_timoutms
,base.json_data:acwSettings.wrapupPrompt::varchar(100) as acwsettings_wrapupprompt
,base.json_data:skillEvaluationMethod::varchar(100) as skillEvaluationMethod

,base.json_data:queueFlow.id::varchar(100) as queueflow_id
,base.json_data:queueFlow.name::varchar(100) as queueflow_name
,base.json_data:queueFlow.selfUri::varchar(100) as queueflow_selfuri

,base.json_data:whisperPrompt.id::varchar(100) as whisperprompt_id
,base.json_data:whisperPrompt.name::varchar(100) as whisperprompt_name
,base.json_data:whisperPrompt.selfUri::varchar(100) as whisperprompt_selfuri
,base.json_data:autoAnswerOnly as autoansweronly
,base.json_data:callingPartyName::varchar(100) as callingPartyName
,base.json_data:callingPartyNumber::varchar(100) as callingPartyNumber

,base.json_data:defaultScripts.CALL.id::varchar(100) as defaultscripts_call_id
,base.json_data:defaultScripts.CALL.selfUri::varchar(100) as defaultscripts_call_selfuri
,base.json_data:defaultScripts.EMAIL.id::varchar(100) as defaultscripts_email_id
,base.json_data:defaultScripts.EMAIL.selfUri::varchar(100) as defaultscripts_email_selfuri

,base.json_data:suppressInQueueCallRecording as suppressInQueueCallRecording
,base.json_data:selfUri::varchar(100) as selfuri
from genesys.raw.queue_json base

In [ ]:
utc_now=datetime.now(pytz.utc)#current time in utc time
# print(utc_now)
end_time = utc_now.replace(minute=0, second=0, microsecond=0)+timedelta(hours=1)#current utc time rounded UP to the nearest hour. we round this to an hour because we will merge the data based on the interval timestamps later on, so we would not want any minutes showing up in the data based on the time of the run.
# print(end_time)
start_time = end_time - timedelta(days=1)
interval=f"{start_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')}/{end_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')}"
# interval="2024-09-26T00:00:00/2024-09-27T00:00:00"
# interval="2024-09-27T00:00:00/2024-09-28T00:00:00"
# interval="2024-09-28T00:00:00/2024-09-29T00:00:00"
# interval="2024-09-29T00:00:00/2024-09-30T00:00:00"
# interval="2024-09-30T00:00:00/2024-10-01T00:00:00"
# interval="2024-10-01T00:00:00/2024-10-02T00:00:00"
# interval="2024-08-01T00:00:00/2024-08-02T00:00:00"
# interval="2024-09-02T00:00:00/2024-09-03T00:00:00"
# interval="2024-09-03T00:00:00/2024-09-04T00:00:00"

# interval="2024-09-04T00:00:00/2024-09-05T00:00:00"
# interval="2024-09-05T00:00:00/2024-08-06T00:00:00"
# interval="2024-09-06T00:00:00/2024-08-07T00:00:00"
# interval="2024-09-07T00:00:00/2024-08-08T00:00:00"
# interval="2024-09-08T00:00:00/2024-08-09T00:00:00"
# interval="2024-09-09T00:00:00/2024-08-10T00:00:00"
# interval="2024-09-10T00:00:00/2024-08-11T00:00:00"
# interval="2024-09-11T00:00:00/2024-08-12T00:00:00"
# interval="2024-09-12T00:00:00/2024-08-13T00:00:00"
# interval="2024-09-13T00:00:00/2024-08-14T00:00:00"
# interval="2024-09-14T00:00:00/2024-08-15T00:00:00"
# interval="2024-09-15T00:00:00/2024-08-16T00:00:00"
# interval="2024-09-16T00:00:00/2024-08-17T00:00:00"
# interval="2024-09-17T00:00:00/2024-08-18T00:00:00"
# interval="2024-09-18T00:00:00/2024-08-19T00:00:00"
# interval="2024-09-19T00:00:00/2024-08-20T00:00:00"
# interval="2024-09-20T00:00:00/2024-08-21T00:00:00"
# interval="2024-09-21T00:00:00/2024-08-22T00:00:00"
# interval="2024-09-22T00:00:00/2024-08-23T00:00:00"
# interval="2024-09-23T00:00:00/2024-08-24T00:00:00"
# interval="2024-09-24T00:00:00/2024-08-25T00:00:00"
# interval="2024-09-25T00:00:00/2024-08-26T00:00:00"
# interval="2024-09-26T00:00:00/2024-08-27T00:00:00"
# interval="2024-08-27T00:00:00/2024-08-28T00:00:00"
# interval="2024-08-28T00:00:00/2024-08-29T00:00:00"
# interval="2024-08-29T00:00:00/2024-08-30T00:00:00"
# interval="2024-08-30T00:00:00/2024-08-31T00:00:00"
# interval="2024-08-31T00:00:00/2024-09-01T00:00:00"
# interval="2024-09-01T00:00:00/2024-09-02T00:00:00"
print(interval)

In [ ]:

#conversations summary. There are only 70 records for this. it looks like a reference table
def conversation_summary_api(interval):
    table_name='genesys.raw.conversation_summary_json'
    columns='json_data'
    
    #initial endpoint
    page_size=100
    page_number=1
    base_url='https://api.mypurecloud.com'
    endpoint='/api/v2/analytics/conversations/aggregates/query?pageSize={0}&pageNumber={1}'.format(page_size, page_number)
    api_endpoint = '{0}{1}'.format(base_url, endpoint)
    #calc the dates for the start and end of the interval
    utc_now=datetime.now(pytz.utc)#current time in utc time
    # print(utc_now)
    end_time = utc_now.replace(minute=0, second=0, microsecond=0)+timedelta(hours=1)#current utc time rounded UP to the nearest hour. we round this to an hour because we will merge the data based on the interval timestamps later on, so we would not want any minutes showing up in the data based on the time of the run.
    # print(end_time)
    start_time = end_time - timedelta(days=1)#start time is 1 day before the end time. we are going to run this job every 30 mins. setting it for 1 day pull from the api each time so that we cover any gaps in execution. will merge the data in the final step
    # print(start_time)
    # interval=f"{start_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')}/{end_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')}"
    #backdating with these intervals manually
    # interval="2024-08-01T00:00:00/2024-08-09T00:00:00"
    # interval="2024-08-09T00:00:00/2024-08-18T00:00:00"
    # interval="2024-08-18T00:00:00/2024-08-27T00:00:00"
    # interval="2024-08-27T00:00:00/2024-09-01T00:00:00"
    # interval="2024-09-01T00:00:00/2024-09-09T00:00:00"
    # interval="2024-09-09T00:00:00/2024-09-15T00:00:00"
    # interval="2024-09-15T00:00:00/2024-09-20T00:00:00"
    
    # interval="2024-09-20T00:00:00/2024-09-26T00:00:00"
    # interval="2024-09-15T00:00:00/2024-09-20T00:00:00"
    
    # interval="2024-09-20T00:00:00/2024-09-26T00:00:00"
    # print(interval)
    #query to pass to the api
    query_payload = {
      
    "interval": interval,
      "groupBy": [
        "userId",
          "queueid"
      ],
      "metrics": [
        "nOffered",
        "nOutbound",
        "nOutboundAbandoned",
        "nConnected",
        "nTransferred",
        "nOverSla",
        "oServiceLevel",
        "oServiceTarget",
        "tAcd",
        "tAbandon",
        "tAcw",
        "tHandle",
        "tHeld",
        "tWait",
        "tTalk",
        "tAnswered",
          "tAlert",
    "tNotResponding"
      ],
      "granularity": "PT30M"#"P1DT1H" #this is the time interval to group by
    }
    
    #flag to stop pagination
    keep_paginating=True 
    #list to collect all data from each pagination api hit
    all_conversation_summary_data=[]
    #pagination loop
    while keep_paginating:
        conversation_summary_data=fetch_data(api_endpoint, auth_token, query_payload)
        print(conversation_summary_data)
        results=conversation_summary_data['results']
        df=pandas.DataFrame({'json_data':results})
        snowflake_df=session.create_dataframe(df)
        #truncate staging table and load data
        session.sql(f"truncate table {table_name}").collect()
        snowflake_df.write.mode("append").save_as_table(table_name)
        all_conversation_summary_data.extend(conversation_summary_data['results'])# i dont think this is relevant here
        if 'nextUri' in conversation_summary_data.keys():
            api_endpoint = '{0}{1}'.format(base_url, conversation_summary_data['nextUri'])
            print('wowowowowowowowowoowowowowow there is pagination')
        else:
            keep_paginating=False  
# print(all_conversation_summary_data)
# print(len(all_conversation_summary_data))
conversation_summary_api(interval)

In [ ]:
--staging table for the structured data
create or replace table genesys.raw.conversation_summary_staged as
with long_data as (

select distinct 
metric.value as metric_object,
json_data:group:userId as userid
,json_data:group:queueId as queueId 
,json_data:group:mediaType as mediatype 
,data.value:interval as interval
from genesys.raw.conversation_summary_json base, 
lateral flatten (input=>base.json_data:data)  data ,
lateral flatten (input=> data.value:metrics)  metric
)
,key_pair_data as (
select userid, queueid, mediatype, interval
, metric_object:metric || '_count' as metric_name 
, metric_object:stats.count as metric_stat
from long_data
where metric_object:metric in ('tAcd','tWait','tHandle','tHeld', 'tTalk', 'tAcw','tAbandon','tAnswered','tAlert','tNotResponding' )
union 
select userid, queueid, mediatype, interval
, metric_object:metric || '_sum' as metric_name 
, metric_object:stats.sum as metric_stat
from long_data
where metric_object:metric in ('tAcd','tWait','tHandle','tHeld', 'tTalk', 'tAcw','tAbandon','tAnswered','tAlert','tNotResponding')
union 
select userid, queueid, mediatype, interval
, metric_object:metric || '_max' as metric_name 
, metric_object:stats.max as metric_stat
from long_data
where metric_object:metric in ('tAcd','tWait','tHandle','tHeld', 'tTalk', 'tAcw','tAbandon','tAnswered','tAlert','tNotResponding')
union 
select userid, queueid, mediatype, interval
, metric_object:metric || '_min' as metric_name 
, metric_object:stats.min as metric_stat
from long_data
where metric_object:metric in ('tAcd','tWait','tHandle','tHeld', 'tTalk', 'tAcw','tAbandon','tAnswered','tAlert','tNotResponding')
union 
select userid, queueid, mediatype, interval
, metric_object:metric || '_target' as metric_name 
, metric_object:stats.target as metric_stat
from long_data
where metric_object:metric = 'oServiceLevel'
union 
select userid, queueid, mediatype, interval
, metric_object:metric || '_current' as metric_name 
, metric_object:stats.current as metric_stat
from long_data
where metric_object:metric = 'oServiceTarget'
union 
select userid, queueid, mediatype, interval
, metric_object:metric || '_count' as metric_name 
, metric_object:stats.count as metric_stat
from long_data
where metric_object:metric not in ('oServiceTarget','oServiceLevel','tAcd','tWait','tHandle','tHeld', 'tTalk', 'tAcw','tAbandon','tAnswered')
)

, flat_data as (
select distinct userid, queueId, interval, mediatype
,max(case when metric_name='tAcd_count' then metric_stat else null end) as tAcd_count
,max(case when metric_name='tWait_count' then metric_stat else null end) as tWait_count
,max(case when metric_name='tHandle_count' then metric_stat else null end) as tHandle_count
,max(case when metric_name='tHeld_count' then metric_stat else null end) as tHeld_count
,max(case when metric_name='tTalk_count' then metric_stat else null end) as tTalk_count
,max(case when metric_name='tAcw_count' then metric_stat else null end) as tAcw_count
,max(case when metric_name='tAbandon_count' then metric_stat else null end) as tAbandon_count
,max(case when metric_name='tAnswered_count' then metric_stat else null end) as tAnswered_count
,max(case when metric_name='tAlert_count' then metric_stat else null end) as tAlert_count
,max(case when metric_name='tNotResponding_count' then metric_stat else null end) as tNotResponding_count

,max(case when metric_name='tAcd_sum' then metric_stat else null end) as tAcd_sum
,max(case when metric_name='tWait_sum' then metric_stat else null end) as tWait_sum
,max(case when metric_name='tHandle_sum' then metric_stat else null end) as tHandle_sum
,max(case when metric_name='tHeld_sum' then metric_stat else null end) as tHeld_sum
,max(case when metric_name='tTalk_sum' then metric_stat else null end) as tTalk_sum
,max(case when metric_name='tAcw_sum' then metric_stat else null end) as tAcw_sum
,max(case when metric_name='tAbandon_sum' then metric_stat else null end) as tAbandon_sum
,max(case when metric_name='tAnswered_sum' then metric_stat else null end) as tAnswered_sum
,max(case when metric_name='tAlert_sum' then metric_stat else null end) as tAlert_sum
,max(case when metric_name='tNotResponding_sum' then metric_stat else null end) as tNotResponding_sum

,max(case when metric_name='tAcd_max' then metric_stat else null end) as tAcd_max
,max(case when metric_name='tWait_max' then metric_stat else null end) as tWait_max
,max(case when metric_name='tHandle_max' then metric_stat else null end) as tHandle_max
,max(case when metric_name='tHeld_max' then metric_stat else null end) as tHeld_max
,max(case when metric_name='tTalk_max' then metric_stat else null end) as tTalk_max
,max(case when metric_name='tAcw_max' then metric_stat else null end) as tAcw_max
,max(case when metric_name='tAbandon_max' then metric_stat else null end) as tAbandon_max
,max(case when metric_name='tAnswered_max' then metric_stat else null end) as tAnswered_max

,max(case when metric_name='tAcd_min' then metric_stat else null end) as tAcd_min
,max(case when metric_name='tWait_min' then metric_stat else null end) as tWait_min
,max(case when metric_name='tHandle_min' then metric_stat else null end) as tHandle_min
,max(case when metric_name='tHeld_min' then metric_stat else null end) as tHeld_min
,max(case when metric_name='tTalk_min' then metric_stat else null end) as tTalk_min
,max(case when metric_name='tAcw_min' then metric_stat else null end) as tAcw_min
,max(case when metric_name='tAbandon_min' then metric_stat else null end) as tAbandon_min
,max(case when metric_name='tAnswered_min' then metric_stat else null end) as tAnswered_min

,max(case when metric_name='nOffered_count' then metric_stat else null end) as nOffered_count
,max(case when metric_name='nOutbound_count' then metric_stat else null end) as nOutbound_count
,max(case when metric_name='nOutboundAbandoned_count' then metric_stat else null end) as nOutboundAbandoned_count
,max(case when metric_name='nConnected_count' then metric_stat else null end) as nConnected_count
,max(case when metric_name='nTransferred_count' then metric_stat else null end) as nTransferred_count
,max(case when metric_name='nOverSla_count' then metric_stat else null end) as nOverSla_count

,max(case when metric_name='oServiceLevel_target' then metric_stat else null end) as oServiceLevel_target
,max(case when metric_name='oServiceTarget_current' then metric_stat else null end) as oServiceTarget_current

from key_pair_data 
group by userid, queueId, interval, mediatype
)
-- select * from key_pair_data;
select userid::varchar(100) as userid
, queueid::varchar(100) as queueid
, interval
, split_part(interval, '/',1)::timestamp_ntz as start_interval
, split_part(interval, '/',2)::timestamp_ntz as end_interval
, mediatype::varchar(100) as mediatype

, tacd_count::int as tacd_count
, tacd_sum::int as tacd_sum
, tacd_max::int as tacd_max
, tacd_min::int as tacd_min

, twait_count::int as twait_count
, twait_sum::int as twait_sum
, twait_max::int as twait_max
, twait_min::int as twait_min

, thandle_count::int as thandle_count
, thandle_sum::int as thandle_sum
, thandle_max::int as thandle_max
, thandle_min::int as thandle_min

, theld_count::int as theld_count
, theld_sum::int as theld_sum
, theld_max::int as theld_max
, theld_min::int as theld_min

, ttalk_count::int as ttalk_count
, ttalk_sum::int as ttalk_sum
, ttalk_max::int as ttalk_max
, ttalk_min::int as ttalk_min

, tacw_count::int as tacw_count
, tacw_sum::int as tacw_sum
, tacw_max::int as tacw_max
, tacw_min::int as tacw_min

, tabandon_count::int as tabandon_count
, tabandon_sum::int as tabandon_sum
, tabandon_max::int as tabandon_max
, tabandon_min::int as tabandon_min

, tanswered_count::int as tanswered_count
, tanswered_sum::int as tanswered_sum
, tanswered_max::int as tanswered_max
, tanswered_min::int as tanswered_min

, talert_count::int as talert_count
, talert_sum::int as talert_sum

, tNotResponding_count::int as tNotResponding_count
, tNotResponding_sum::int as tNotResponding_sum


, noffered_count::int as noffered_count
, noutbound_count::int as noutbound_count
, noutboundabandoned_count::int as noutboundabandoned_count
, nconnected_count::int as nconnected_count
, ntransferred_count::int as ntransferred_count
, noversla_count::int as noversla_count

, oservicelevel_target::float as oServiceLevel_target
, oservicetarget_current::float as OServiceTarget_current
from flat_data

In [ ]:
merge into genesys.transform.conversation_summary s
using genesys.raw.conversation_summary_staged t on ifnull(s.userid,'xxx') = ifnull(t.userid,'xxx') and ifnull(s.queueid,'xxx') = ifnull(t.queueid,'xxx') and ifnull(s.interval,'xxx')=ifnull(t.interval,'xxx') and ifnull(s.mediatype,'xxx')=ifnull(t.mediatype,'xxx')
when not matched then insert (
    USERID,
    QUEUEID,
    INTERVAL,
    START_INTERVAL,
    END_INTERVAL,
    MEDIATYPE,
    
    TACD_count,
    TACD_sum,
    TACD_max,
    TACD_min,

    TWAIT_count,
    TWAIT_sum,
    TWAIT_max,
    TWAIT_min,
    
    THANDLE_count,
    THANDLE_sum,
    THANDLE_max,
    THANDLE_min,
    
    THELD_count,
    THELD_sum,
    THELD_max,
    THELD_min,
    
    TTALK_count,
    TTALK_sum,
    TTALK_max,
    TTALK_min,
    
    TACW_count,
    TACW_sum,
    TACW_max,
    TACW_min,
    
    TABANDON_count,
    TABANDON_sum,
    TABANDON_max,
    TABANDON_min,
    
    TANSWERED_count,
    TANSWERED_sum,
    TANSWERED_max,
    TANSWERED_min,

    TALERT_count,
    TALERT_sum,

    tNotResponding_count,
    tNotResponding_sum,
    
    NOFFERED_count,
    NOUTBOUND_count,
    NOUTBOUNDABANDONED_count,
    NCONNECTED_count,
    NTRANSFERRED_count,
    NOVERSLA_count,
  OSERVICELEVEL_target,
  OSERVICETARGET_current,
    CREATED_TIMESTAMP, --CREATED DATE
    UPDATED_TIMESTAMP --UPDATED DATE
) values (
    t.USERID,
    t.QUEUEID,
    t.INTERVAL,
    t.START_INTERVAL,
    t.END_INTERVAL,
    t.MEDIATYPE,
    
    t.TACD_count,
    t.TACD_sum,
    t.TACD_max,
    t.TACD_min,
    
    t.TWAIT_count,
    t.TWAIT_sum,
    t.TWAIT_max,
    t.TWAIT_min,
    
    t.THANDLE_count,
    t.THANDLE_sum,
    t.THANDLE_max,
    t.THANDLE_min,
    
    t.THELD_count,
    t.THELD_sum,
    t.THELD_max,
    t.THELD_min,
    
    t.TTALK_count,
    t.TTALK_sum,
    t.TTALK_max,
    t.TTALK_min,
    
    t.TACW_count,
    t.TACW_sum,
    t.TACW_max,
    t.TACW_min,
    
    t.TABANDON_count,
    t.TABANDON_sum,
    t.TABANDON_max,
    t.TABANDON_min,
    
    t.TANSWERED_count,
    t.TANSWERED_sum,
    t.TANSWERED_max,
    t.TANSWERED_min,

    t.TALERT_COUNT,
    t.TALERT_SUM,

    t.tNotResponding_count,
    t.tNotResponding_sum,
    
    t.NOFFERED_count,
    t.NOUTBOUND_count,
    t.NOUTBOUNDABANDONED_count,
    t.NCONNECTED_count,
    t.NTRANSFERRED_count,
    t.NOVERSLA_count,
  t.OSERVICELEVEL_target,
  t.OSERVICETARGET_current,
     CURRENT_TIMESTAMP, --CREATED TIMESTAMP
    CURRENT_TIMESTAMP --UPDATED TIMESTAMP
)
when matched then update set 
    s.USERID=t.USERID,
    s.QUEUEID=t.QUEUEID,
    s.INTERVAL=t.INTERVAL,
    s.START_INTERVAL=t.START_INTERVAL,
    s.END_INTERVAL=t.END_INTERVAL,
    s.MEDIATYPE=t.MEDIATYPE,
    
    s.TACD_count=t.TACD_count,
    s.TACD_sum=t.TACD_sum,
    s.TACD_max=t.TACD_max,
    s.TACD_min=t.TACD_min,
    
    s.TWAIT_count=t.TWAIT_count,
    s.TWAIT_sum=t.TWAIT_sum,
    s.TWAIT_max=t.TWAIT_max,
    s.TWAIT_min=t.TWAIT_min,
    
    s.THANDLE_count=t.THANDLE_count,
    s.THANDLE_sum=t.THANDLE_sum,
    s.THANDLE_max=t.THANDLE_max,
    s.THANDLE_min=t.THANDLE_min,
    
    s.THELD_count=t.THELD_count,
    s.THELD_sum=t.THELD_sum,
    s.THELD_max=t.THELD_max,
    s.THELD_min=t.THELD_min,
    
    s.TTALK_count=t.TTALK_count,
    s.TTALK_sum=t.TTALK_sum,
    s.TTALK_max=t.TTALK_max,
    s.TTALK_min=t.TTALK_min,
    
    s.TACW_count=t.TACW_count,
    s.TACW_sum=t.TACW_sum,
    s.TACW_max=t.TACW_max,
    s.TACW_min=t.TACW_min,
    
    s.TABANDON_count=t.TABANDON_count,
    s.TABANDON_sum=t.TABANDON_sum,
    s.TABANDON_max=t.TABANDON_max,
    s.TABANDON_min=t.TABANDON_min,
    
    s.TANSWERED_count=t.TANSWERED_count,
    s.TANSWERED_sum=t.TANSWERED_sum,
    s.TANSWERED_max=t.TANSWERED_max,
    s.TANSWERED_min=t.TANSWERED_min,

    s.talert_count=t.talert_count,
    s.talert_sum=t.talert_sum,

    s.tNotResponding_count= t.tNotResponding_count,
    s.tNotResponding_sum= t.tNotResponding_sum,
    
    s.NOFFERED_count=t.NOFFERED_count,
    s.NOUTBOUND_count=t.NOUTBOUND_count,
    s.NOUTBOUNDABANDONED_count=t.NOUTBOUNDABANDONED_count,
    s.NCONNECTED_count=t.NCONNECTED_count,
    s.NTRANSFERRED_count=t.NTRANSFERRED_count,
    s.NOVERSLA_count=t.NOVERSLA_count,
  s.OSERVICELEVEL_target=t.OSERVICELEVEL_target,
  s.OSERVICETARGET_current=t.OSERVICETARGET_current,
    s.UPDATED_TIMESTAMP = current_timestamp;

In [ ]:
#not actively running as this was not needed for first wave of reporting. UPDATE, we do need this for reporting now
#conversations detail
def conversation_detail_api(interval):


    print('startingggggggg')
    table_name='genesys.raw.conversation_detail_json'
    columns='json_data'
    
    #initial endpoint
    page_size=100
    page_number=1
    base_url='https://api.mypurecloud.com'
    endpoint='/api/v2/analytics/conversations/details/query'
    api_endpoint = '{0}{1}'.format(base_url, endpoint)
    # interval="2024-09-23T00:00:00/2024-09-24T00:00:00"
    # interval="2024-09-24T00:00:00/2024-09-25T00:00:00"
    # interval="2024-09-25T00:00:00/2024-09-26T00:00:00"
    # interval="2024-09-26T00:00:00/2024-09-27T00:00:00"
    # interval="2024-09-27T00:00:00/2024-09-28T00:00:00"
    # interval="2024-09-20T00:00:00/2024-09-21T00:00:00"
    print(interval)
    #query to pass to the api
    query_payload = {
      "interval": interval,
        "paging": {
            "pageSize": page_size,
            "pageNumber": page_number
        }
      
    }
    
    #flag to stop pagination
    keep_paginating=True 
    #list to collect all data from each pagination api hit
    all_conversation_summary_data=[]
    #pagination loop
    counter=1
    key_list=[]
    session.sql(f"truncate table {table_name}").collect()
    while keep_paginating:
        conversation_summary_data=fetch_data(api_endpoint, auth_token, query_payload)
        conversations=conversation_summary_data['conversations']
        record_count=conversation_summary_data['totalHits']
        df=pandas.DataFrame({'json_data':conversations})
        snowflake_df=session.create_dataframe(df)
        
        #write to snowflake
        snowflake_df.write.mode("append").save_as_table(table_name)
        
        if record_count/page_size>page_number:
            counter=counter+1
            print(counter)
            page_number=counter
            #pagination handling in the query payload for this endpoint
            query_payload = {
      "interval": interval,
        "paging": {
            "pageSize": page_size,
            "pageNumber": page_number
        }
      
    }
        else:
            keep_paginating=False

conversation_detail_api(interval)

In [ ]:
create or replace table genesys.raw.conversation_detail_staged as 
SELECT distinct base.json_data:conversationId::varchar(100) as conversationId
, base.json_data:conversationStart::timestamp_ntz as conversationStart
, base.json_data:conversationEnd::timestamp_ntz as conversationEnd
 FROM genesys.raw.CONVERSATION_DETAIL_JSON base 

In [ ]:
merge into genesys.transform.conversation_detail s
using genesys.raw.conversation_detail_staged t on ifnull(s.conversationid,'xxx') = ifnull(t.conversationid,'xxx') 
when not matched then insert (
    conversationid,
    conversationstart,
    conversationend,
    CREATED_TIMESTAMP, --CREATED DATE
    UPDATED_TIMESTAMP --UPDATED DATE
) values (
    t.conversationid,
    t.conversationstart,
    t.conversationend,
     CURRENT_TIMESTAMP, --CREATED TIMESTAMP
    CURRENT_TIMESTAMP --UPDATED TIMESTAMP
)
when matched then update set 
    s.conversationstart=t.conversationstart,
    s.conversationend=t.conversationend,
    s.UPDATED_TIMESTAMP = current_timestamp

In [ ]:
create or replace table genesys.raw.conversation_detail_segments_staged as 


SELECT distinct base.json_data:conversationId::varchar(100) as conversationId
, participants.value:purpose::varchar(25) as purpose
, participants.value:userId::varchar(100) as userId
, sessions.value:sessionId::varchar(100) as sessionId
, segments.value:segmentStart::timestamp_ntz as segmentStart
, segments.value:segmentEnd::timestamp_ntz as segmentEnd
, segments.value:segmentType::varchar(25) as segmentType
, segments.value:conference::boolean as conference
, segments.value:queueId::varchar(100) as queueId
, segments.value:disconnectType::varchar(25) as disconnectType
, segments.value:errorCode::varchar(1000) as errorCode
, segments.value:wrapUpCode::varchar(100) as wrapUpCode
 FROM genesys.raw.CONVERSATION_DETAIL_JSON base ,
lateral flatten (input=>base.json_data:participants) participants,
lateral flatten (input=> participants.value:sessions)  sessions,
lateral flatten (input=> sessions.value:segments)  segments
order by 2

In [ ]:
merge into genesys.transform.conversation_detail_segments s
using genesys.raw.conversation_detail_segments_staged t on ifnull(s.conversationid,'xxx') = ifnull(t.conversationid,'xxx') 
    and ifnull(s.sessionid,'xxx') = ifnull(t.sessionid,'xxx')
    and s.segmentstart = t.segmentstart
    and s.segmentend = t.segmentend
    and ifnull(s.segmenttype,'xxx') = ifnull(t.segmenttype,'xxx')
    and ifnull(s.userId,'xxx') = ifnull(t.userId,'xxx')
    
when not matched then insert (
    conversationid,
    purpose,
    userId,
    sessionId,
    segmentStart,
    segmentEnd,
    segmentType,
    conference,
    queueId,
    disconnectType,
    errorCode,
    wrapUpCode,
    CREATED_TIMESTAMP, --CREATED DATE
    UPDATED_TIMESTAMP --UPDATED DATE
) values (
    t.conversationid,
    t.purpose,
    t.userId,
    t.sessionId,
    t.segmentStart,
    t.segmentEnd,
    t.segmentType,
    t.conference,
    t.queueId,
    t.disconnectType,
    t.errorCode,
    t.wrapUpCode,
     CURRENT_TIMESTAMP, --CREATED TIMESTAMP
    CURRENT_TIMESTAMP --UPDATED TIMESTAMP
)
when matched then update set 
    s.purpose=t.purpose,
    s.queueId=t.queueId,
    s.conference=t.conference,
    s.disconnectType=t.disconnectType,
    s.errorCode=t.errorCode,
    s.wrapUpCode=t.wrapUpCode,
    s.UPDATED_TIMESTAMP = current_timestamp

In [ ]:
# interval_list=["2024-09-20T00:00:00/2024-09-21T00:00:00","2024-09-21T00:00:00/2024-09-22T00:00:00"]


# for interval in interval_list:
#     conversation_summary_api(interval)
#     session.sql(f"""create or replace table genesys.raw.conversation_summary_staged as
#     with long_data as (
    
#     select distinct 
#     metric.value as metric_object,
#     json_data:group:userId as userid
#     ,json_data:group:queueId as queueId 
#     ,json_data:group:mediaType as mediatype 
#     ,data.value:interval as interval
#     from genesys.raw.conversation_summary_json base, 
#     lateral flatten (input=>base.json_data:data)  data ,
#     lateral flatten (input=> data.value:metrics)  metric
#     )
#     ,key_pair_data as (
#     select userid, queueid, mediatype, interval
#     , metric_object:metric || '_count' as metric_name 
#     , metric_object:stats.count as metric_stat
#     from long_data
#     where metric_object:metric in ('tAcd','tWait','tHandle','tHeld', 'tTalk', 'tAcw','tAbandon','tAnswered')
#     union 
#     select userid, queueid, mediatype, interval
#     , metric_object:metric || '_sum' as metric_name 
#     , metric_object:stats.sum as metric_stat
#     from long_data
#     where metric_object:metric in ('tAcd','tWait','tHandle','tHeld', 'tTalk', 'tAcw','tAbandon','tAnswered')
#     union 
#     select userid, queueid, mediatype, interval
#     , metric_object:metric || '_max' as metric_name 
#     , metric_object:stats.max as metric_stat
#     from long_data
#     where metric_object:metric in ('tAcd','tWait','tHandle','tHeld', 'tTalk', 'tAcw','tAbandon','tAnswered')
#     union 
#     select userid, queueid, mediatype, interval
#     , metric_object:metric || '_min' as metric_name 
#     , metric_object:stats.min as metric_stat
#     from long_data
#     where metric_object:metric in ('tAcd','tWait','tHandle','tHeld', 'tTalk', 'tAcw','tAbandon','tAnswered')
#     union 
#     select userid, queueid, mediatype, interval
#     , metric_object:metric || '_target' as metric_name 
#     , metric_object:stats.target as metric_stat
#     from long_data
#     where metric_object:metric = 'oServiceLevel'
#     union 
#     select userid, queueid, mediatype, interval
#     , metric_object:metric || '_current' as metric_name 
#     , metric_object:stats.current as metric_stat
#     from long_data
#     where metric_object:metric = 'oServiceTarget'
#     union 
#     select userid, queueid, mediatype, interval
#     , metric_object:metric || '_count' as metric_name 
#     , metric_object:stats.count as metric_stat
#     from long_data
#     where metric_object:metric not in ('oServiceTarget','oServiceLevel','tAcd','tWait','tHandle','tHeld', 'tTalk', 'tAcw','tAbandon','tAnswered')
#     )
    
#     , flat_data as (
#     select distinct userid, queueId, interval, mediatype
#     ,max(case when metric_name='tAcd_count' then metric_stat else null end) as tAcd_count
#     ,max(case when metric_name='tWait_count' then metric_stat else null end) as tWait_count
#     ,max(case when metric_name='tHandle_count' then metric_stat else null end) as tHandle_count
#     ,max(case when metric_name='tHeld_count' then metric_stat else null end) as tHeld_count
#     ,max(case when metric_name='tTalk_count' then metric_stat else null end) as tTalk_count
#     ,max(case when metric_name='tAcw_count' then metric_stat else null end) as tAcw_count
#     ,max(case when metric_name='tAbandon_count' then metric_stat else null end) as tAbandon_count
#     ,max(case when metric_name='tAnswered_count' then metric_stat else null end) as tAnswered_count
    
#     ,max(case when metric_name='tAcd_sum' then metric_stat else null end) as tAcd_sum
#     ,max(case when metric_name='tWait_sum' then metric_stat else null end) as tWait_sum
#     ,max(case when metric_name='tHandle_sum' then metric_stat else null end) as tHandle_sum
#     ,max(case when metric_name='tHeld_sum' then metric_stat else null end) as tHeld_sum
#     ,max(case when metric_name='tTalk_sum' then metric_stat else null end) as tTalk_sum
#     ,max(case when metric_name='tAcw_sum' then metric_stat else null end) as tAcw_sum
#     ,max(case when metric_name='tAbandon_sum' then metric_stat else null end) as tAbandon_sum
#     ,max(case when metric_name='tAnswered_sum' then metric_stat else null end) as tAnswered_sum
    
#     ,max(case when metric_name='tAcd_max' then metric_stat else null end) as tAcd_max
#     ,max(case when metric_name='tWait_max' then metric_stat else null end) as tWait_max
#     ,max(case when metric_name='tHandle_max' then metric_stat else null end) as tHandle_max
#     ,max(case when metric_name='tHeld_max' then metric_stat else null end) as tHeld_max
#     ,max(case when metric_name='tTalk_max' then metric_stat else null end) as tTalk_max
#     ,max(case when metric_name='tAcw_max' then metric_stat else null end) as tAcw_max
#     ,max(case when metric_name='tAbandon_max' then metric_stat else null end) as tAbandon_max
#     ,max(case when metric_name='tAnswered_max' then metric_stat else null end) as tAnswered_max
    
#     ,max(case when metric_name='tAcd_min' then metric_stat else null end) as tAcd_min
#     ,max(case when metric_name='tWait_min' then metric_stat else null end) as tWait_min
#     ,max(case when metric_name='tHandle_min' then metric_stat else null end) as tHandle_min
#     ,max(case when metric_name='tHeld_min' then metric_stat else null end) as tHeld_min
#     ,max(case when metric_name='tTalk_min' then metric_stat else null end) as tTalk_min
#     ,max(case when metric_name='tAcw_min' then metric_stat else null end) as tAcw_min
#     ,max(case when metric_name='tAbandon_min' then metric_stat else null end) as tAbandon_min
#     ,max(case when metric_name='tAnswered_min' then metric_stat else null end) as tAnswered_min
    
#     ,max(case when metric_name='nOffered_count' then metric_stat else null end) as nOffered_count
#     ,max(case when metric_name='nOutbound_count' then metric_stat else null end) as nOutbound_count
#     ,max(case when metric_name='nOutboundAbandoned_count' then metric_stat else null end) as nOutboundAbandoned_count
#     ,max(case when metric_name='nConnected_count' then metric_stat else null end) as nConnected_count
#     ,max(case when metric_name='nTransferred_count' then metric_stat else null end) as nTransferred_count
#     ,max(case when metric_name='nOverSla_count' then metric_stat else null end) as nOverSla_count
    
#     ,max(case when metric_name='oServiceLevel_target' then metric_stat else null end) as oServiceLevel_target
#     ,max(case when metric_name='oServiceTarget_current' then metric_stat else null end) as oServiceTarget_current
    
#     from key_pair_data 
#     group by userid, queueId, interval, mediatype
#     )
#     -- select * from key_pair_data;
#     select userid::varchar(100) as userid
#     , queueid::varchar(100) as queueid
#     , interval
#     , split_part(interval, '/',1)::timestamp_ntz as start_interval
#     , split_part(interval, '/',2)::timestamp_ntz as end_interval
#     , mediatype::varchar(100) as mediatype
    
#     , tacd_count::int as tacd_count
#     , tacd_sum::int as tacd_sum
#     , tacd_max::int as tacd_max
#     , tacd_min::int as tacd_min
    
#     , twait_count::int as twait_count
#     , twait_sum::int as twait_sum
#     , twait_max::int as twait_max
#     , twait_min::int as twait_min
    
#     , thandle_count::int as thandle_count
#     , thandle_sum::int as thandle_sum
#     , thandle_max::int as thandle_max
#     , thandle_min::int as thandle_min
    
#     , theld_count::int as theld_count
#     , theld_sum::int as theld_sum
#     , theld_max::int as theld_max
#     , theld_min::int as theld_min
    
#     , ttalk_count::int as ttalk_count
#     , ttalk_sum::int as ttalk_sum
#     , ttalk_max::int as ttalk_max
#     , ttalk_min::int as ttalk_min
    
#     , tacw_count::int as tacw_count
#     , tacw_sum::int as tacw_sum
#     , tacw_max::int as tacw_max
#     , tacw_min::int as tacw_min
    
#     , tabandon_count::int as tabandon_count
#     , tabandon_sum::int as tabandon_sum
#     , tabandon_max::int as tabandon_max
#     , tabandon_min::int as tabandon_min
    
#     , tanswered_count::int as tanswered_count
#     , tanswered_sum::int as tanswered_sum
#     , tanswered_max::int as tanswered_max
#     , tanswered_min::int as tanswered_min
    
#     , noffered_count::int as noffered_count
#     , noutbound_count::int as noutbound_count
#     , noutboundabandoned_count::int as noutboundabandoned_count
#     , nconnected_count::int as nconnected_count
#     , ntransferred_count::int as ntransferred_count
#     , noversla_count::int as noversla_count
    
#     , oservicelevel_target::float as oServiceLevel_target
#     , oservicetarget_current::float as OServiceTarget_current
#     from flat_data""").collect()
#     session.sql(f"""merge into genesys.transform.conversation_summary s
#     using genesys.raw.conversation_summary_staged t on ifnull(s.userid,'xxx') = ifnull(t.userid,'xxx') and ifnull(s.queueid,'xxx') = ifnull(t.queueid,'xxx') and ifnull(s.interval,'xxx')=ifnull(t.interval,'xxx') and ifnull(s.mediatype,'xxx')=ifnull(t.mediatype,'xxx')
#     when not matched then insert (
#         USERID,
#         QUEUEID,
#         INTERVAL,
#         START_INTERVAL,
#         END_INTERVAL,
#         MEDIATYPE,
        
#         TACD_count,
#         TACD_sum,
#         TACD_max,
#         TACD_min,
    
#         TWAIT_count,
#         TWAIT_sum,
#         TWAIT_max,
#         TWAIT_min,
        
#         THANDLE_count,
#         THANDLE_sum,
#         THANDLE_max,
#         THANDLE_min,
        
#         THELD_count,
#         THELD_sum,
#         THELD_max,
#         THELD_min,
        
#         TTALK_count,
#         TTALK_sum,
#         TTALK_max,
#         TTALK_min,
        
#         TACW_count,
#         TACW_sum,
#         TACW_max,
#         TACW_min,
        
#         TABANDON_count,
#         TABANDON_sum,
#         TABANDON_max,
#         TABANDON_min,
        
#         TANSWERED_count,
#         TANSWERED_sum,
#         TANSWERED_max,
#         TANSWERED_min,
        
#         NOFFERED_count,
#         NOUTBOUND_count,
#         NOUTBOUNDABANDONED_count,
#         NCONNECTED_count,
#         NTRANSFERRED_count,
#         NOVERSLA_count,
#       OSERVICELEVEL_target,
#       OSERVICETARGET_current,
#         CREATED_TIMESTAMP, --CREATED DATE
#         UPDATED_TIMESTAMP --UPDATED DATE
#     ) values (
#         t.USERID,
#         t.QUEUEID,
#         t.INTERVAL,
#         t.START_INTERVAL,
#         t.END_INTERVAL,
#         t.MEDIATYPE,
        
#         t.TACD_count,
#         t.TACD_sum,
#         t.TACD_max,
#         t.TACD_min,
        
#         t.TWAIT_count,
#         t.TWAIT_sum,
#         t.TWAIT_max,
#         t.TWAIT_min,
        
#         t.THANDLE_count,
#         t.THANDLE_sum,
#         t.THANDLE_max,
#         t.THANDLE_min,
        
#         t.THELD_count,
#         t.THELD_sum,
#         t.THELD_max,
#         t.THELD_min,
        
#         t.TTALK_count,
#         t.TTALK_sum,
#         t.TTALK_max,
#         t.TTALK_min,
        
#         t.TACW_count,
#         t.TACW_sum,
#         t.TACW_max,
#         t.TACW_min,
        
#         t.TABANDON_count,
#         t.TABANDON_sum,
#         t.TABANDON_max,
#         t.TABANDON_min,
        
#         t.TANSWERED_count,
#         t.TANSWERED_sum,
#         t.TANSWERED_max,
#         t.TANSWERED_min,
        
#         t.NOFFERED_count,
#         t.NOUTBOUND_count,
#         t.NOUTBOUNDABANDONED_count,
#         t.NCONNECTED_count,
#         t.NTRANSFERRED_count,
#         t.NOVERSLA_count,
#       t.OSERVICELEVEL_target,
#       t.OSERVICETARGET_current,
#          CURRENT_TIMESTAMP, --CREATED TIMESTAMP
#         CURRENT_TIMESTAMP --UPDATED TIMESTAMP
#     )
#     when matched then update set 
#         s.USERID=t.USERID,
#         s.QUEUEID=t.QUEUEID,
#         s.INTERVAL=t.INTERVAL,
#         s.START_INTERVAL=t.START_INTERVAL,
#         s.END_INTERVAL=t.END_INTERVAL,
#         s.MEDIATYPE=t.MEDIATYPE,
        
#         s.TACD_count=t.TACD_count,
#         s.TACD_sum=t.TACD_sum,
#         s.TACD_max=t.TACD_max,
#         s.TACD_min=t.TACD_min,
        
#         s.TWAIT_count=t.TWAIT_count,
#         s.TWAIT_sum=t.TWAIT_sum,
#         s.TWAIT_max=t.TWAIT_max,
#         s.TWAIT_min=t.TWAIT_min,
        
#         s.THANDLE_count=t.THANDLE_count,
#         s.THANDLE_sum=t.THANDLE_sum,
#         s.THANDLE_max=t.THANDLE_max,
#         s.THANDLE_min=t.THANDLE_min,
        
#         s.THELD_count=t.THELD_count,
#         s.THELD_sum=t.THELD_sum,
#         s.THELD_max=t.THELD_max,
#         s.THELD_min=t.THELD_min,
        
#         s.TTALK_count=t.TTALK_count,
#         s.TTALK_sum=t.TTALK_sum,
#         s.TTALK_max=t.TTALK_max,
#         s.TTALK_min=t.TTALK_min,
        
#         s.TACW_count=t.TACW_count,
#         s.TACW_sum=t.TACW_sum,
#         s.TACW_max=t.TACW_max,
#         s.TACW_min=t.TACW_min,
        
#         s.TABANDON_count=t.TABANDON_count,
#         s.TABANDON_sum=t.TABANDON_sum,
#         s.TABANDON_max=t.TABANDON_max,
#         s.TABANDON_min=t.TABANDON_min,
        
#         s.TANSWERED_count=t.TANSWERED_count,
#         s.TANSWERED_sum=t.TANSWERED_sum,
#         s.TANSWERED_max=t.TANSWERED_max,
#         s.TANSWERED_min=t.TANSWERED_min,
        
#         s.NOFFERED_count=t.NOFFERED_count,
#         s.NOUTBOUND_count=t.NOUTBOUND_count,
#         s.NOUTBOUNDABANDONED_count=t.NOUTBOUNDABANDONED_count,
#         s.NCONNECTED_count=t.NCONNECTED_count,
#         s.NTRANSFERRED_count=t.NTRANSFERRED_count,
#         s.NOVERSLA_count=t.NOVERSLA_count,
#       s.OSERVICELEVEL_target=t.OSERVICELEVEL_target,
#       s.OSERVICETARGET_current=t.OSERVICETARGET_current,
#         s.UPDATED_TIMESTAMP = current_timestamp""").collect()
    
#     print(interval)
#     print('iamhere')
#     conversation_detail_api(interval)
#     session.sql(f"""create or replace table genesys.raw.conversation_detail_staged as 
#     SELECT distinct base.json_data:conversationId::varchar(100) as conversationId
#     , base.json_data:conversationStart::timestamp_ntz as conversationStart
#     , base.json_data:conversationEnd::timestamp_ntz as conversationEnd
#      FROM genesys.raw.CONVERSATION_DETAIL_JSON base""").collect()
#     session.sql(f"""merge into genesys.transform.conversation_detail s
#     using genesys.raw.conversation_detail_staged t on ifnull(s.conversationid,'xxx') = ifnull(t.conversationid,'xxx') 
#     when not matched then insert (
#         conversationid,
#         conversationstart,
#         conversationend,
#         CREATED_TIMESTAMP, --CREATED DATE
#         UPDATED_TIMESTAMP --UPDATED DATE
#     ) values (
#         t.conversationid,
#         t.conversationstart,
#         t.conversationend,
#          CURRENT_TIMESTAMP, --CREATED TIMESTAMP
#         CURRENT_TIMESTAMP --UPDATED TIMESTAMP
#     )
#     when matched then update set 
#         s.conversationstart=t.conversationstart,
#         s.conversationend=t.conversationend,
#         s.UPDATED_TIMESTAMP = current_timestamp""").collect()
#     session.sql(f"""create or replace table genesys.raw.conversation_detail_segments_staged as 
    
    
#     SELECT distinct base.json_data:conversationId::varchar(100) as conversationId
#     , participants.value:purpose::varchar(25) as purpose
#     , participants.value:userId::varchar(100) as userId
#     , sessions.value:sessionId::varchar(100) as sessionId
#     , segments.value:segmentStart::timestamp_ntz as segmentStart
#     , segments.value:segmentEnd::timestamp_ntz as segmentEnd
#     , segments.value:segmentType::varchar(25) as segmentType
#     , segments.value:conference::boolean as conference
#     , segments.value:queueId::varchar(100) as queueId
#     , segments.value:disconnectType::varchar(25) as disconnectType
#     , segments.value:errorCode::varchar(1000) as errorCode
#     , segments.value:wrapUpCode::varchar(100) as wrapUpCode
#      FROM genesys.raw.CONVERSATION_DETAIL_JSON base ,
#     lateral flatten (input=>base.json_data:participants) participants,
#     lateral flatten (input=> participants.value:sessions)  sessions,
#     lateral flatten (input=> sessions.value:segments)  segments
#     order by 2""").collect()
#     session.sql(f"""merge into genesys.transform.conversation_detail_segments s
#     using genesys.raw.conversation_detail_segments_staged t on ifnull(s.conversationid,'xxx') = ifnull(t.conversationid,'xxx') 
#         and ifnull(s.sessionid,'xxx') = ifnull(t.sessionid,'xxx')
#         and s.segmentstart = t.segmentstart
#         and s.segmentend = t.segmentend
#         and ifnull(s.segmenttype,'xxx') = ifnull(t.segmenttype,'xxx')
#         and ifnull(s.userId,'xxx') = ifnull(t.userId,'xxx')
        
#     when not matched then insert (
#         conversationid,
#         purpose,
#         userId,
#         sessionId,
#         segmentStart,
#         segmentEnd,
#         segmentType,
#         conference,
#         queueId,
#         disconnectType,
#         errorCode,
#         wrapUpCode,
#         CREATED_TIMESTAMP, --CREATED DATE
#         UPDATED_TIMESTAMP --UPDATED DATE
#     ) values (
#         t.conversationid,
#         t.purpose,
#         t.userId,
#         t.sessionId,
#         t.segmentStart,
#         t.segmentEnd,
#         t.segmentType,
#         t.conference,
#         t.queueId,
#         t.disconnectType,
#         t.errorCode,
#         t.wrapUpCode,
#          CURRENT_TIMESTAMP, --CREATED TIMESTAMP
#         CURRENT_TIMESTAMP --UPDATED TIMESTAMP
#     )
#     when matched then update set 
#         s.purpose=t.purpose,
#         s.queueId=t.queueId,
#         s.conference=t.conference,
#         s.disconnectType=t.disconnectType,
#         s.errorCode=t.errorCode,
#         s.wrapUpCode=t.wrapUpCode,
#         s.UPDATED_TIMESTAMP = current_timestamp""").collect()



In [ ]:
# interval_list=["2024-09-20T00:00:00/2024-09-21T00:00:00","2024-09-21T00:00:00/2024-09-22T00:00:00"]
# for interval in interval_list:
#     conversation_detail_api(interval)

# interval_list=["2024-09-20T00:00:00/2024-09-21T00:00:00","2024-09-21T00:00:00/2024-09-22T00:00:00"]

# for interval in interval_list:
#     print(interval)
#     conversation_detail_api(interval)
#     session.sql(f"""create or replace table genesys.raw.conversation_detail_staged as 
#     SELECT distinct base.json_data:conversationId::varchar(100) as conversationId
#     , base.json_data:conversationStart::timestamp_ntz as conversationStart
#     , base.json_data:conversationEnd::timestamp_ntz as conversationEnd
#      FROM genesys.raw.CONVERSATION_DETAIL_JSON base""").collect()
#     session.sql(f"""merge into genesys.transform.conversation_detail s
#     using genesys.raw.conversation_detail_staged t on ifnull(s.conversationid,'xxx') = ifnull(t.conversationid,'xxx') 
#     when not matched then insert (
#         conversationid,
#         conversationstart,
#         conversationend,
#         CREATED_TIMESTAMP, --CREATED DATE
#         UPDATED_TIMESTAMP --UPDATED DATE
#     ) values (
#         t.conversationid,
#         t.conversationstart,
#         t.conversationend,
#          CURRENT_TIMESTAMP, --CREATED TIMESTAMP
#         CURRENT_TIMESTAMP --UPDATED TIMESTAMP
#     )
#     when matched then update set 
#         s.conversationstart=t.conversationstart,
#         s.conversationend=t.conversationend,
#         s.UPDATED_TIMESTAMP = current_timestamp""").collect()
#     session.sql(f"""create or replace table genesys.raw.conversation_detail_segments_staged as 
    
    
#     SELECT distinct base.json_data:conversationId::varchar(100) as conversationId
#     , participants.value:purpose::varchar(25) as purpose
#     , participants.value:userId::varchar(100) as userId
#     , sessions.value:sessionId::varchar(100) as sessionId
#     , segments.value:segmentStart::timestamp_ntz as segmentStart
#     , segments.value:segmentEnd::timestamp_ntz as segmentEnd
#     , segments.value:segmentType::varchar(25) as segmentType
#     , segments.value:conference::boolean as conference
#     , segments.value:queueId::varchar(100) as queueId
#     , segments.value:disconnectType::varchar(25) as disconnectType
#     , segments.value:errorCode::varchar(1000) as errorCode
#     , segments.value:wrapUpCode::varchar(100) as wrapUpCode
#      FROM genesys.raw.CONVERSATION_DETAIL_JSON base ,
#     lateral flatten (input=>base.json_data:participants) participants,
#     lateral flatten (input=> participants.value:sessions)  sessions,
#     lateral flatten (input=> sessions.value:segments)  segments
#     order by 2""").collect()
#     session.sql(f"""merge into genesys.transform.conversation_detail_segments s
#     using genesys.raw.conversation_detail_segments_staged t on ifnull(s.conversationid,'xxx') = ifnull(t.conversationid,'xxx') 
#         and ifnull(s.sessionid,'xxx') = ifnull(t.sessionid,'xxx')
#         and s.segmentstart = t.segmentstart
#         and s.segmentend = t.segmentend
#         and ifnull(s.segmenttype,'xxx') = ifnull(t.segmenttype,'xxx')
#         and ifnull(s.userId,'xxx') = ifnull(t.userId,'xxx')
        
#     when not matched then insert (
#         conversationid,
#         purpose,
#         userId,
#         sessionId,
#         segmentStart,
#         segmentEnd,
#         segmentType,
#         conference,
#         queueId,
#         disconnectType,
#         errorCode,
#         wrapUpCode,
#         CREATED_TIMESTAMP, --CREATED DATE
#         UPDATED_TIMESTAMP --UPDATED DATE
#     ) values (
#         t.conversationid,
#         t.purpose,
#         t.userId,
#         t.sessionId,
#         t.segmentStart,
#         t.segmentEnd,
#         t.segmentType,
#         t.conference,
#         t.queueId,
#         t.disconnectType,
#         t.errorCode,
#         t.wrapUpCode,
#          CURRENT_TIMESTAMP, --CREATED TIMESTAMP
#         CURRENT_TIMESTAMP --UPDATED TIMESTAMP
#     )
#     when matched then update set 
#         s.purpose=t.purpose,
#         s.queueId=t.queueId,
#         s.conference=t.conference,
#         s.disconnectType=t.disconnectType,
#         s.errorCode=t.errorCode,
#         s.wrapUpCode=t.wrapUpCode,
#         s.UPDATED_TIMESTAMP = current_timestamp""").collect()